### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# cell for work
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
# cell for work
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0111161,10,1373234211
1,1,0117060,7,1373415231
2,1,0120755,6,1373424360
3,1,0317919,6,1373495763
4,1,0454876,10,1373621125


In [18]:
movies.shape

(34328, 3)

In [19]:
reviews.shape

(812217, 4)

In [26]:
# more work
genres = []

for val in movies['genre']:
#     print(val)
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass
    
unique_genres = set(genres)
print(len(unique_genres))
unique_genres

28


{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [30]:
# and still more
len(np.unique(reviews.user_id))

60116

In [32]:
# and even more - open additional cells as necessary
reviews.rating.isnull().sum()

0

In [34]:
reviews.rating.mean()

7.3045984016586702

In [35]:
reviews.rating.min()

0

In [36]:
reviews.rating.max()

10

In [40]:
reviews.rating.describe()

count    812217.000000
mean          7.304598
std           1.855791
min           0.000000
25%           6.000000
50%           7.000000
75%           9.000000
max          10.000000
Name: rating, dtype: float64

In [ ]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 34328,
'The number of ratings in the dataset': 812217,
'The number of different genres': 28,
'The number of unique users in the dataset': 60116,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.3045984016586702,
'The minimum rating given across all ratings': 0,
'The maximum rating given across all ratings': 10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [42]:
# cell for work
create_date = lambda x : x[-5:-1] if x[-1] == ')' else np.nan
movies['date'] = movies['movie'].apply(create_date)
movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [44]:
# and another
def create_century(val):
    if val[:2] == yr:
        return 1
    else:
        return 0

for yr in ['18','19', '20']:
    movies[yr+"00's"] = movies['date'].apply(create_century)
    
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [50]:
# and a few more below, which you can use as necessary
def create_dummies_for_genre(val):
    try:
        if val.find(gnr) > -1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0
    
for gnr in unique_genres:
    movies[gnr]= movies['genre'].apply(create_dummies_for_genre)
    
movies.head()


,movie_id,movie,genre,date,1800's,1900's,2000's,Western,Thriller,Sport,...,Biography,Game-Show,History,Music,Adult,Comedy,Film-Noir,Documentary,Musical,Fantasy
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [53]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0111161,10,1373234211,2013-07-07 21:56:51
1,1,0117060,7,1373415231,2013-07-10 00:13:51
2,1,0120755,6,1373424360,2013-07-10 02:46:00
3,1,0317919,6,1373495763,2013-07-10 22:36:03
4,1,0454876,10,1373621125,2013-07-12 09:25:25


In [54]:
# now reviews and movies are the final dataframes with the necessary columns
reviews.to_csv('./reviews_clean.csv')
movies.to_csv('./movies_clean.csv')

In [ ]:
reviews_new, movies_new = t.show_clean_dataframes()